** See congested Bus Trips! **

The app predicts congestion in bus trips, uploads the result to our Elastic DB, and displays the journeys (journeyPatternId) with congestion on the map below.
Missing data and intregration of external info is done automatically!

All you need to do is specifiy the data path (or IP if streaming), select the filters you want, and press the update button! 

*More detailed explanation below.

In [2]:
from IPython.display import Image
Image(url="https://drive.google.com/uc?id=1S13K-zCiK2nLgY9pdSSjgntxyOl-nm-x",  width=150)

In [3]:
!pip install libify
import libify
lab4_functions = libify.importer(globals(), '/Users/shubi@campus.technion.ac.il/Lab4_functions')

dbutils.widgets.dropdown("Is Weekend?", "True", ['True', 'False'])
dbutils.widgets.dropdown("Passes a Popular Location?", "True", ['True', 'False'])
dbutils.widgets.dropdown("Is Event Date?", "True", ['True', 'False'])
dbutils.widgets.dropdown("Part of Day (Hours)", "4-7", ['0-3',  '4-7', '8-11', '12-15', '16-19', '20-23'])
dbutils.widgets.dropdown("Stream?", "False", ['True', 'False'])
dbutils.widgets.text("IP / Path", '')

isWeekend = eval(dbutils.widgets.get("Is Weekend?"))
isPassedLocation = eval(dbutils.widgets.get("Passes a Popular Location?"))
isEventDate = eval(dbutils.widgets.get("Is Event Date?"))
hourGroup = dbutils.widgets.get("Part of Day (Hours)")
isStream = eval(dbutils.widgets.get("Stream?"))
path_ip = dbutils.widgets.get("IP / Path")

params={"isWeekend":isWeekend, "isEventDate":isEventDate, "isPassedLocation":isPassedLocation, "hourGroup":hourGroup}
# print(params, isStream, path_ip) # todo delete
# Our check
# 10.0.0.30:9091
# /mnt/dacoursedatabricksstg/dacoursedatabricksdata/busFile 
test_predictions = lab4_functions.raw_data_to_predictions(streaming=isStream, USE_TAKE=False, min_date=None, max_date=None, filter_params=params, path_ip=path_ip)

if test_predictions == 'problem':
    print('\n\n\n\n')
    print('Unexpected error in reading file.\n'
          'Please make sure the file exists, ' 
          'is of the correct format(json) and with the right schema.')
else:
    journey_patterns = test_predictions.where(test_predictions.prediction==1).select('journeyPatternId')
    lab4_functions.write_to_elastic(journey_patterns, 'journey-patterns-with-congestion', lab4_functions.elastic_settings_mappings_basic, overwrite=True, streaming=isStream)

displayHTML("""<iframe src="https://da2020w-0001.eastus.cloudapp.azure.com:5601/goto/b956cced0cd51ab36979d7f2852d2f93" height="450" width="1024"></iframe>""")

* Choose the data to run on by adding ip number if it is a streaming data, and a file path if it is a batch data. and select the right value in the 'Stram?'' bullet. 
* Select what to filter the bus trips on using the following bullets:
  * Is Event Date? - if True predict congestion on bus trips accured in a day with big event (e.g. day with football game).
  * Passes a Popular Location? - if True predict congestion on bus trips passed near popular location\locations (like Temple Bar, Dublin, Ireland).
  * Is Weekend? - if True predict congestion on bus trips accured in the weekend.
  * Part of Day - predict congestion on bus trips accured in the chosen hours of the day.
* Run by pressing on the "update" button that appears at the top right of the app. (It might pop-up a massage, please press 'yes')
* -> The map automaticaly will be updated! On the map you will see plots - each point color represent some Journy Pattern Id, and by hoover on the point you can see the Journy Pattern number.
* Notes: 
  1. If the map is empty there are no Journy Pattern Ids with congestion with the chosen filters or you might entered a wrong ip number.
  2. The running might take a while.
  3. If you see a pop-up massage "Run beyond time out" press it to continue.
  4. if you are getting error "Failed to construct kafka consumer" - you might entered a wrong ip number.